In [1]:
import os
import json
from ast import literal_eval
from collections import defaultdict

In [2]:
databases = 'C:/Users/Chris-Brota/Desktop/Databases'
display(os.listdir(databases))

# Se elige el JSON
db = f'{databases}/{os.listdir(databases)[1]}'

['foodb_2020_04_07_csv',
 'foodb_2020_04_07_json',
 'foodb_server_dump_2020_4_21.sql',
 'FoodData_Central_foundation_food_csv_2021-04-28']

# Revisión de componentes de las bebidas energéticas

In [65]:
# Poblando el diccionario de información
energyzer_fold = 'bebidas'
energyzer_files = [f'{energyzer_fold}/{i}' for i in os.listdir(energyzer_fold)]
energyzer_compounds = defaultdict(list)

for name in energyzer_files:
    with open(name, 'r', encoding='utf8') as file:
        value_total = 0
        
        # Obtener la dosis
        dosis = file.readline().strip().split(';')
        
        # Saltar la primera linea
        file.readline()
        
        # Agregar los nombres
        energyzer_compounds['"Name"'].append(name)
        
        # Modificando la dosis total
        if dosis[1] == 'g':
            dosis_total = float(dosis[0])
        elif dosis[1] == 'mg':
            dosis_total = float(dosis[0]) / 1e3
        elif dosis[1] == 'ug':
            dosis_total = float(dosis[0]) / 1e6
        
        # Agregando la dosis al registro
        energyzer_compounds['"Size"'].append(dosis_total)
        
        # Revisando los datos
        for line in file:
            data_line = line.strip().split(';')
            
            # Obtención del gramaje
            gram_unit = data_line[2].lower()
            
            # Modificando el gramaje
            if gram_unit == 'g':
                value_to = float(data_line[1])
            elif gram_unit == 'mg':
                value_to = float(data_line[1]) / 1e3
            elif gram_unit == 'ug':
                value_to = float(data_line[1]) / 1e6
            
            # Normalizando por la dosis total
            value_perc = value_to / dosis_total
            
            # Agregando al diccionario
            energyzer_compounds[data_line[0]].append(value_perc)

In [38]:
# Definición de las componentes que aportan
compounds_key = list()

for key in energyzer_compounds.keys():
    if key != '"Name"':
        # Si es que al menos tiene algo que aportar, se agrega
        if sum(energyzer_compounds[key]) > 0:
            compounds_key.append(key)
            print(key)

"Fat"
"Protein"
"Carbohydrate"
"Water"
"Caffeine"
"Sugars"
"Calcium, Ca"
"Iron, Fe"
"Magnesium, Mg"
"Phosphorus, P"
"Potassium, K"
"Sodium, Na"
"Zinc, Zn"
"Copper, Cu"
"Selenium, Se"
"Vitamin C"
"Thiamin"
"Riboflavin"
"Niacin"
"Vitamin B6"
"Folate"
"Vitamin B12"
"Choline"
"Vitamin K"
"Folic acid"
"Folate, DFE"
"Vitamin B12, added"
"Net carbs"


### Test de sanidad por bebida

In [62]:
for num, name in enumerate(energyzer_compounds['"Name"']):
    keys_to_rev = list(energyzer_compounds.keys())[2:]
    
    # Cantidad de gramos por componentes
    grams_drink = 0
    
    for key in keys_to_rev:
        grams_drink += energyzer_compounds[key][num]
        
    print(name, grams_drink)
    

bebidas/energy_drink.csv 1.2047139687500001
bebidas/energy_drink_full_throttle.csv 1.2396013457258066
bebidas/energy_drink_monster.csv 1.2218857612903222
bebidas/energy_drink_mountain_dew_amp.csv 1.239678411532258
bebidas/energy_drink_nos.csv 1.2232145096774196
bebidas/energy_drink_no_fear.csv 1.288126075241936
bebidas/energy_drink_no_fear_motherload.csv 1.288126075241936
bebidas/energy_drink_ocean_spray_cranenergy_juice_drink.csv 1.0750881629032256
bebidas/energy_drink_red_bull.csv 1.2047139687500001
bebidas/energy_drink_rockstar.csv 1.2537157891129032
bebidas/energy_drink_sobe_energize_energy_juice_drink.csv 1.2147129032258066
bebidas/energy_drink_vault.csv 1.2574499254032256
bebidas/energy_drink_xs.csv 1.0022780176612904
bebidas/energy_drink_xs_gold_plus.csv 1.0022780176612904


# Revisando alimentos energéticos por descripción

In [60]:
with open(f'{db}/Food.json', 'r', encoding='utf8') as file:
    # Definición de la lista de alimentos energéticos 
    energetic_foods = list()
    
    keyword = 'energ'
    
    # Para cada línea en el archivo
    for line in file:
        data = json.loads(line.strip())
        
        if data['description'] is not None:
            if keyword in data['description']:
                energetic_foods.append((data['id'], data['name'], data['public_id']))

In [61]:
energetic_foods

[(246, 'Triticale', 'FOOD00246'),
 (670, 'Sugar', 'FOOD00651'),
 (676, 'Sugar substitute', 'FOOD00656'),
 (727, 'Energy drink', 'FOOD00703'),
 (779, 'Nutritional drink', 'FOOD00754'),
 (848, 'Trail mix', 'FOOD00819'),
 (893, 'Milk and milk products', 'FOOD00863'),
 (905, 'Root vegetables', 'FOOD00873'),
 (947, 'Guarana', 'FOOD00915'),
 (982, 'Cottage cheese', 'FOOD00950')]

In [65]:
energetic_list = ['FOOD00336', 'FOOD00336', 'FOOD00168', 'FOOD00058', 'FOOD00572', 
                  'FOOD00068', 'FOOD00367', 'FOOD00219', 'FOOD00723', 'FOOD00206', 
                  'FOOD00555', 'FOOD00109', 'FOOD00379', 'FOOD00159', 'FOOD00647', 
                  'FOOD00165', 'FOOD00548', 'FOOD00183', 'FOOD00040']

# Agregando las encontradas anteriormente
energetic_list.extend(['FOOD00246', 'FOOD00873', 'FOOD00915'])

# Revisando alimentos energéticos por nutrientes o principios activos

In [67]:
# Definición de la lista de componentes
compound = ['FDB003010', 'FDB009020', 'FDB000571', 'FDB016592', 'FDB013228', 'FDB012292', 
            'FDB014504', 'FDB001108', 'FDB023903', 'FDB016251', 'FDB000572', 'FDB012738', 
            'FDB013567', 'FDB003518', 'FDB024414', 'FDB003521', 'FDB031212', 'FDB011795', 
            'FDB003523', 'FDB003191', 'FDB000455', 'FDB000453', 'FDB014601', 'FDB014797', 
            'FDB002251', 'FDB022939', 'FDB001014', 'FDB000574', 'FDB017534', 'FDB023125', 
            'FDB001977', 'FDB003729']

In [79]:
# Pasando a número
with open(f'{db}/Compound.json', 'r', encoding='utf8') as file:
    # Definición de la lista de las componentes
    compound_num = list()
    
    # Para cada línea en el archivo
    for line in file:
        data = json.loads(line.strip())
        
        if data['public_id'] in compound:
            compound_num.append((data['id'], data['public_id']))

In [90]:
comp_only_num = [i[0] for i in compound_num]
print(comp_only_num)

[453, 455, 571, 572, 574, 1014, 1108, 1977, 2251, 3011, 3192, 3522, 3524, 3730, 9021, 11798, 12295, 12741, 13231, 13570, 14507, 14604, 14800, 16258, 16599, 17541, 23102, 23288, 24066, 24577, 31375]


In [91]:
energetic_list_ppo = list()

# Revisando ahora en la base de contenidos
with open(f'{db}/Content.json', 'r', encoding='utf8') as file:
    # Definición de la lista de alimentos energéticos 
    energetic_foods_num = list()
    
#     i = 0
    # Para cada línea en el archivo
    for line in file:
        data = json.loads(line.strip())
        
#         print(data)
#         i+=1
#         if i > 1:
#             break
            
        if data['source_id'] in comp_only_num:
            energetic_foods_num.append(data['food_id'])

In [94]:
len(set(energetic_foods_num))

967

# Obtener las tablas de cada alimento